## Algolia - CRM-система со встроенным ранжированием поисков пользователей. 

Используется в:
- https://www.highsnobiety.com/en-us/shop/category/footwear/sneakers/
- https://www.kickscrew.com/collections/

Search api reference: https://www.algolia.com/doc/api-reference/api-methods/search

### highsnobiety
Доступные категории:

- category/footwear/sneakers - все кроссовки
- brand/adidas/sneakers - по бренду

### kickscrew
Доступные категории:

- TODO
- TODO
 

In [1]:
# !pip install algoliasearch pandas beautifulsoup

import os
import requests
import boto3
import pandas as pd

from tqdm import tqdm
from algoliasearch.search_client import SearchClient
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from typing import Callable

In [2]:
class ParsePipeline:

    def __init__(self, website_name, bucket_name, parser: Callable):
        self.WEBSITE_NAME = website_name
        self.parser = parser
        self.unique_items = set()
        self.bucket_name = bucket_name
        self.s3 = boto3.session.Session().client(
            service_name='s3',
            endpoint_url='https://storage.yandexcloud.net'
        )

    @staticmethod
    def get_image_extension(url: str) -> str:
        return '.' + url[-5:].split('.')[-1]

    # def save_to_filesystem(self, brand: str, name: str, image_url: str) -> str:
    #     photo_folder = os.path.join(self.WEBSITE_NAME, "photos", brand)
    #     os.makedirs(photo_folder, exist_ok=True)
    # 
    #     image_path = os.path.join(photo_folder, name)
    #     img_binary = requests.get(image_url).content
    # 
    #     os.makedirs(photo_folder, exist_ok=True)
    #     with open(image_path, "wb") as f:
    #         f.write(img_binary)
    #     return image_path

    def upload_to_s3(self, brand: str, name: str, image_url: str) -> str:
        img_binary = requests.get(image_url).content
        image_path = os.path.join('data', self.WEBSITE_NAME, brand, name)
        self.s3.put_object(Bucket=self.bucket_name, Key=image_path, Body=img_binary)
        print(f"uploaded: {image_path}")
        return image_path

    def process_item(self, item: dict) -> dict:
        return self.parser(item)

    def parse(self, filters: str, pages: int = 10, hit_per_page: int = 100) -> list:
        full_metadata = []
        for page in range(1, pages):
            resp = index.search('', {
                'filters': filters,
                'page': page,
                'hitsPerPage': hit_per_page
            })
            print(f'processing page {page} with {len(resp["hits"])} items')
            for hit in resp['hits']:
                print(f'Processing brand: {hit["displayBrand"]}')
                metadata = self.process_item(hit)
                if metadata['url'] in self.unique_items:
                    print(metadata['title'], 'is already parsed')
                    continue

                full_metadata.append(metadata)
                self.unique_items.add(metadata['title'])
        print(f"finished parse with {len(full_metadata)} items")
        return full_metadata

    def parse_more_images(self, metadata: dict):
        if metadata.get('right-side-img') is not None:
            print(f"skipped {metadata['slug']}")
            return

        print(f"processing {metadata['slug']}")
        r = requests.get(metadata['url'], headers={"User-Agent": UserAgent().random})
        try:
            r.raise_for_status()
        except Exception as e:
            print(f"could not request page {metadata['url']} - {e}")
            return

        soup = BeautifulSoup(r.text, "html.parser")
        metadata['description'] = soup.select('#acc-controls-details-editorsnote > div > p')[0].text

        image_side = {
            'right-side-img': '#pdp_image_zoom_0',
            'left-side-img': '#pdp_image_zoom_1',
            'front-both-img': '#pdp_image_zoom_2'
        }
        for side, tag in image_side.items():
            t = soup.select(tag)
            if len(t) > 0:
                img_link = t[0]['srcset'].split(', ')[0].split()[0]
                img_path = self.upload_to_s3(  # can be replaced on self.save_to_filesystem
                    metadata['brand'],
                    f"{metadata['slug']}-{side}{self.get_image_extension(img_link)}",
                    img_link
                )
                metadata[side] = img_path
        print(f"{metadata['slug']} is done")


In [3]:
#
# highsnobiety
# 
client = SearchClient.create('KV0RT3PJZC', '37e783b9d54217566682938fa0e2b0f6')
index = client.init_index('production_ec_products_improved_newest_first')
WEBSITE_NAME = 'highsnobiety'


def highsnobiety_parser(item: dict) -> dict:
    us_price = item['price'].get('US', {})
    return {
        'url': f'https://www.highsnobiety.com/en-us/shop/product/{item["slug"]["en"]}',
        'brand': item['displayBrand'],
        'slug': item["slug"]["en"],
        'priceCurrency': us_price['currencyCode'],
        'price': int(us_price.get('centAmount', 0)) / 100,
        'title': item.get('displayName', {}).get('en').replace('\\u', '')
    }


parser = ParsePipeline(WEBSITE_NAME, 'sneakers-ml', highsnobiety_parser)
parse_result = parser.parse(
    'localizedPages.en:category/footwear/sneakers AND outOfStockForLongTime:false',
    10,
    100
)


processing page 1 with 100 items
Processing brand: HOKA
Processing brand: HOKA
Processing brand: HOKA
Processing brand: New Balance
Processing brand: New Balance
Processing brand: New Balance
Processing brand: Salomon
Processing brand: Salomon
Processing brand: Adidas
Processing brand: Converse
Processing brand: Saucony
Processing brand: Saucony
Processing brand: Saucony
Processing brand: Saucony
Processing brand: asics
Processing brand: Adidas
Processing brand: Adidas
Processing brand: Adidas
Processing brand: On
Processing brand: On
Processing brand: Converse
Processing brand: New Balance
Processing brand: New Balance
Processing brand: New Balance
Processing brand: ROA
Processing brand: ROA
Processing brand: ROA
Processing brand: ROA
Processing brand: Puma
Processing brand: Keen
Processing brand: Keen
Processing brand: Keen x Hiking Patrol
Processing brand: asics x GmbH
Processing brand: asics
Processing brand: asics
Processing brand: asics
Processing brand: Reebok LTD
Processing bra

In [5]:
for metadata in tqdm(parse_result):
    parser.parse_more_images(metadata)


  0%|          | 0/598 [00:00<?, ?it/s]

skipped hoka-bondi-8-olive-haze
skipped hoka-bondi-8-black-black
skipped hoka-restore-tc-black
skipped new-balance-m2002rdq-sandstone
skipped new-balance-m2002rdn-dark-moss
skipped new-balance-ml610xa-team-away-grey
skipped salomon-xt-quest-2-falcon-cement-bright-green
skipped salomon-acs-pro-pewter-monument-aegean-blue
skipped adidas-gazelle-core-black-gum
skipped converse-one-star-pro-ltd-ox-berlin-pink
skipped saucony-progrid-triumph-4-green-silver
skipped saucony-progrid-triumph-4-pink-silver
skipped saucony-progrid-triumph-4-blue-silver
skipped saucony-peregrine-13-st-glade-bronze
skipped asics-gel-kayano-14-pure-silver-pure-silver
skipped adidas-cg-split-stan-smith-beige-tonecore-black
skipped adidas-cg-split-stan-smith-core-blackgranite
skipped adidas-cg-split-stan-smith-whiteblack
skipped on-cloudultra-2-black
skipped on-cloudstratus-3-undyed-white-sand
skipped converse-pl-vulc-pro-ox-forest-shelter-white
skipped new-balance-bbw550sb-sea-salt-mercury-blue
skipped new-balance-mt

  5%|▌         | 32/598 [00:02<00:39, 14.16it/s]

uploaded: data/highsnobiety/Keen x Hiking Patrol/keen-hiking-patrol-jasper-rocks-sp-phantasmal-red-front-both-img.jpg
keen-hiking-patrol-jasper-rocks-sp-phantasmal-red is done
skipped asics-gmbh-gel-kayano-legacy-cilantro-wood-crepe
skipped asics-gel-1130-oyster-gray-clay-gray
skipped asics-gel-1130-steel-gray-sheet-rock
skipped asics-gel-kayano-14-simply-taupe-oatmeal
skipped reebok-ltd-club-c-ltd-white-luxe-leather
skipped bape-highsnobiety-road-sta-express-beige-charcoal
skipped bape-highsnobiety-bape-sta-ivory
skipped puma-clyde-rhuigi-pristine-pristine-inky-blue
skipped puma-clyde-rhuigi-pristine-sedate-gray-white
skipped vans-og-style-93-lx-marshmallow
skipped asics-gel-venture-6-dark-sepia-dark-sepia
skipped veja-v-10-b-mesh-white-nautico
skipped veja-v-10-chrome-free-leather-white-black-sahara
skipped norda-001-yellow
skipped reebok-club-c-grounds-green
skipped puma-velophasis-technisch-white
skipped puma-velophasis-prm-beige
skipped puma-nanamica-clyde-gore-tex-white
skipped p

 40%|███▉      | 237/598 [00:05<00:07, 48.45it/s]

...

uploaded: data/highsnobiety/ROA/roa-khatarina-sneaker-sand-front-both-img.jpg
roa-khatarina-sneaker-sand is done
processing on-cloudventure-ice-kelp
uploaded: data/highsnobiety/On/on-cloudventure-ice-kelp-right-side-img.jpg
uploaded: data/highsnobiety/On/on-cloudventure-ice-kelp-left-side-img.jpg


100%|█████████▉| 597/598 [21:10<00:03,  3.36s/it]

uploaded: data/highsnobiety/On/on-cloudventure-ice-kelp-front-both-img.jpg
on-cloudventure-ice-kelp is done
processing on-the-roger-clubhouse-white-indigo
uploaded: data/highsnobiety/On/on-the-roger-clubhouse-white-indigo-right-side-img.jpg
uploaded: data/highsnobiety/On/on-the-roger-clubhouse-white-indigo-left-side-img.jpg


100%|██████████| 598/598 [21:13<00:00,  2.13s/it]

uploaded: data/highsnobiety/On/on-the-roger-clubhouse-white-indigo-front-both-img.jpg
on-the-roger-clubhouse-white-indigo is done


In [7]:
df = pd.DataFrame(parse_result)
df.shape


(598, 10)

In [11]:
data = df.to_csv(index=False)
parser.s3.put_object(Bucket=parser.bucket_name, Key=f"data/{WEBSITE_NAME}.csv", Body=data)

{'ResponseMetadata': {'RequestId': '0edc47065b47e69c',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'nginx',
   'date': 'Thu, 19 Oct 2023 18:59:28 GMT',
   'content-type': 'application/octet-stream',
   'transfer-encoding': 'chunked',
   'connection': 'keep-alive',
   'keep-alive': 'timeout=60',
   'etag': '"c6b04682c624804aca9c6fa2b4a92cfa"',
   'x-amz-request-id': '0edc47065b47e69c'},
  'RetryAttempts': 0},
 'ETag': '"c6b04682c624804aca9c6fa2b4a92cfa"'}

In [12]:
df['brand'].value_counts()

brand
New Balance                       128
Converse                           49
Adidas                             38
Vans                               31
asics                              29
                                 ... 
Converse x Fragment                 1
Saucony x Colour Plus Companie      1
Adidas x Bad Bunny                  1
Sean Wotherspoon x Adidas           1
Puma x P.A.M.                       1
Name: count, Length: 72, dtype: int64